# [CDAF] Atividade 4

## Nome: Thiago Pádua de Carvalho

## Matrícula: 2020007066

## Referências
- [1] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.ExpectedThreat.html#socceraction.xthreat.ExpectedThreat
- [3] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.get_successful_move_actions.html#socceraction.xthreat.get_successful_move_actions
- [4] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/xT.html

In [9]:
# Importando bibliotecas
from tqdm import tqdm
import numpy as np
import pandas as pd
import socceraction.spadl as spd
from socceraction import xthreat as xt

### LaLiga  p/ SPADL com pré-processamentos

In [10]:
# carregando os eventos
path = f'../data/events/events_Spain.json'
events = pd.read_json(path_or_buf=path)
events

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422
...,...,...,...,...,...,...,...,...,...,...,...,...
628654,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671
628655,7,Acceleration,[{'id': 1801}],122832,"[{'y': 2, 'x': 88}, {'y': 21, 'x': 97}]",2565927,Others on the ball,682,2H,2940.515560,70,253302673
628656,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674
628657,8,Simple pass,[{'id': 1801}],40756,"[{'y': 74, 'x': 8}, {'y': 56, 'x': 9}]",2565927,Pass,675,2H,2943.089232,85,253302698


In [11]:
# pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
events = events.rename(columns={'id': 'event_id', 'eventId': 'type_id', 'subEventId': 'subtype_id',
                                'teamId': 'team_id', 'playerId': 'player_id', 'matchId': 'game_id'})
events['milliseconds'] = events['eventSec'] * 1000
events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

In [12]:
# carregando as partidas, pois vamos saber quais times jogam em casa e fora p/ usar como parametro do SPADL
path = f'../data/matches/matches_Spain.json'
matches = pd.read_json(path_or_buf=path)

In [13]:
# as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
team_matches = []
for i in tqdm(range(len(matches))):
    match = pd.DataFrame(matches.loc[i, 'teamsData']).T
    match['matchId'] = matches.loc[i, 'wyId']
    team_matches.append(match)
team_matches = pd.concat(team_matches).reset_index(drop=True)

100%|██████████| 380/380 [00:00<00:00, 1275.63it/s]


In [15]:
# fazendo a conversão p/ SPADL, padronizando a direção de jogo da esquerda p/ a direita e adicionando os nomes dos tipos de ações
spadl = []
game_ids = events.game_id.unique().tolist()
for g in tqdm(game_ids):
    match_events = events.loc[events.game_id == g]
    match_home_id = team_matches.loc[(team_matches.matchId == g) & (team_matches.side == 'home'), 'teamId'].values[0]
    match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
    match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
    match_actions = spd.add_names(match_actions)
    spadl.append(match_actions)
spadl = pd.concat(spadl).reset_index(drop=True)

100%|██████████| 380/380 [02:46<00:00,  2.28it/s]


In [25]:
# adicionando o nome dos jogadores
path = f'../data/players.json'
players = pd.read_json(path_or_buf=path)
players['player_name'] = players['firstName'] + ' ' + players['lastName']
players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})
spadl = spadl.merge(players, on='player_id', how='left')

## Questão 1
- Crei um dataframe "shots" à partir do dataframe "spadl", contendo apenas os chutes.
- Crie 4 colunas no dataframe "shots" a serem usadas como features de um modelo de xG.
- Justifique a escolha das features.

In [27]:
shots = spadl.loc[spadl.type_name == 'shot'].reset_index(drop=True)
shots.head()

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id,type_name,result_name,bodypart_name,player_name_x,player_name_y
0,2565548,1,57.771186,695,225089,97.65,44.88,105.0,34.00,180865315,0,11,0,20,shot,fail,foot,Jos\u00e9 Luis Morales Nogales,Jos\u00e9 Luis Morales Nogales
1,2565548,1,60.727239,695,255738,84.00,27.88,84.0,27.88,180864547,0,11,0,22,shot,fail,foot,Jefferson Andr\u00e9s Lerma Sol\u00eds,Jefferson Andr\u00e9s Lerma Sol\u00eds
2,2565548,1,446.986112,682,37831,92.40,29.24,92.4,29.24,180864486,0,11,0,93,shot,fail,foot,Carlos Arturo Bacca Ahumada,Carlos Arturo Bacca Ahumada
3,2565548,1,488.929113,682,15214,91.35,23.12,105.0,27.20,180864491,0,11,0,96,shot,fail,foot,Antonio Rukavina,Antonio Rukavina
4,2565548,1,948.872079,695,225089,78.75,40.80,105.0,34.00,180864792,0,11,0,178,shot,fail,foot,Jos\u00e9 Luis Morales Nogales,Jos\u00e9 Luis Morales Nogales


,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,result_id,action_id,type_name_x,result_name,bodypart_name,player_name,type_name_y
0,2565548,1,2.994582,682,3542,38.85,26.52,52.50,34.00,180864419,0,1,0,pass,success,foot,Manuel Trigueros Mu\u00f1oz,pass
1,2565548,1,3.137020,682,274435,52.50,34.00,47.25,47.60,180864418,0,1,1,pass,success,foot,Enes \u00dcnal,pass
2,2565548,1,6.709668,682,364860,47.25,47.60,39.90,59.84,180864420,0,1,2,pass,success,foot,Rodrigo Hern\u00e1ndez Cascante,pass
3,2565548,1,8.805497,682,3534,39.90,59.84,33.60,21.08,180864421,0,1,3,pass,success,foot,Jaume Vicent Costa Jord\u00e1,pass
4,2565548,1,14.047492,682,3695,33.60,21.08,32.55,42.84,180864422,0,1,4,pass,success,foot,\u00c1lvaro Gonz\u00e1lez Sober\u00f3n,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473889,2565882,2,2302.047015,675,3306,3.15,44.88,105.00,0.00,250612057,0,3,1275,bad_touch,owngoal,foot,Sergio Ramos Garc\u00eda,bad_touch
473890,2565894,1,864.623285,678,3409,4.20,34.00,105.00,0.00,247223547,0,3,198,bad_touch,owngoal,foot,Mikel San Jos\u00e9 Dom\u00ednguez,bad_touch
473891,2565894,2,488.579847,678,3409,7.35,28.56,105.00,0.00,247224199,0,3,731,bad_touch,owngoal,foot,Mikel San Jos\u00e9 Dom\u00ednguez,bad_touch
473892,2565899,2,466.011971,679,8311,8.40,34.00,0.00,68.00,249586602,0,3,901,bad_touch,owngoal,foot,Stefan Savi\u0107,bad_touch


## Questão 2
- Crie uma coluna numérica binária "goal" no dataframe "shots" indicando se o chute resultou em gol ou não.
- Use regressão logística [1] p/ treinar (.fit(X_train, y_train)) um modelo de xG usando as features criadas na questão 1.
- Use 70% dos dados para treino e 30% para teste.
- Reporte a acurácia do modelo para os conjuntos de treino (.score(X_train, y_train)) e teste (.score(X_test, y_test)).

## Questão 3
- Use o modelo treinado na questão 2 p/ prever a probabilidade de gol de todos os chutes do dataframe "shots". Reporte essas probabilidades no dataframe "shots" em uma coluna "xG".
- Agrupe o dataframe "shots" por "player_name" e reporte a soma dos "goal" e "xG".
- Reporte os 10 jogadores com maior xG.
- Reporte os 10 jogadores com maior diferença de Gols e xG.

## Questão 4 [4]
- Instancie um objeto ExpectedThreat [2] com parâmetros l=25 e w=16.
- Faça o fit do modelo ExpectedThreat com o dataframe "spadl".

## Questão 5
- Crie um dataframe "prog_actions" à partir do dataframe "spadl", contendo apenas as ações de progressão e que são bem-sucedidas [3].
- Use o método rate do objeto ExpectedThreat p/ calcular o valor de cada ação de progressão do dataframe "prog_actions", em uma coluna chamada "action_value".
- Agrupe o dataframe "prog_actions" por "player_name" e reporte a soma dos "action_value".
- Reporte os 10 jogadores com maior "action_value".